# 7. 부산 정보산업 진흥원-사업공고 crawling 프로젝트
#### 1) 라이브러리 로드
#### 2) 사업공고 리스트는 페이징 으로 구분
#### 3) 재귀호출 형태로 함수 생성
    3.1)crawling 할 주소에 페이지징 변수 처리
#### 4) 함수 호출 시작
#### 5) crawling 한 데이터를 csv파일로 저장
#### 6) csv파일을 DataFrame 형태로 읽어온다.
#### 7) stopword(불용어)를 적용해서 WordCloud로 표현하기
#### 8) html 페이지 crawling시 에러
    ▶ 증    상 - 한글깨짐현상
    ▶ 해결방안 - response.encoding = 'UTF-8'
 #### 9) csv파일 읽을시 에러발생
    ▶ 증     상 - error 코드 - UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb0 in position 0: invalid start byte
    ▶ 해결방안 - encoding='CP949')
    1차 입니다.


In [33]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져온다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱한다.
from bs4 import BeautifulSoup as bs

In [34]:
    base_url = f"http://busanit.or.kr/board/list.asp?bcode=notice&sword=&search_txt="
    response = requests.get( base_url )
    response.encoding = 'UTF-8'
    soup = bs(response.text, 'html.parser')
    # 소스코드를 제대로 가져왔는지 확인해 봅니다.
    tbodys = soup.select("#sub_cont > div.content_sub > table > tbody")

    for tr in tbodys:
        tds = list(tr.find_all('td'))
        i = 1
        for td in tds:
            i=i+1
            #print(td)
            if td.find('a'):
                title       = td.find('a').text 
                current_img = td.find('img')
                img_src = current_img.get('src')
                img_name = img_src.split('/')[-1]
                
                reg_date    = tds[i-1].text
                view_count  = tds[i].text
                #print(img_name, title, reg_date, view_count)
                #print(img_name, title, reg_date, view_count)


In [35]:
def get_notice_list(ipage):
    base_url = f"http://busanit.or.kr/board/list.asp?bcode=notice&sword=&search_txt=&ipage={ipage}"
    response = requests.get( base_url )
    
    print(response.status_code)
    if response.status_code == 200:
        response.encoding = 'UTF-8'
        soup = bs(response.text, 'html.parser')
        # 소스코드를 제대로 가져왔는지 확인해 봅니다.
        tbodys = soup.select("#sub_cont > div.content_sub > table > tbody")
        for tr in tbodys:
            tds = list(tr.find_all('td'))
            i = 1
            for td in tds:
                i = i+1
                if td.find('a'):
                    current_img = td.find('img')
                    img_src     = current_img.get('src')
                    status      = img_src.split('/')[-1] # 사업공고 상태(진행중-ing / 종료-end)
                    
                    title       = td.find('a').text 
                    title       = title.replace(',','_')
                    
                    reg_date    = tds[i-1].text
                    view_count  = tds[i].text
                    notice_list.append([status, title, reg_date, view_count])    # data 리스트에 제목, 등록일 추가
        ipage += 1
        if ipage < 3:
            # 페이지번호를 하나씩 더해서 재귀호출을 합니다.
            get_notice_list(ipage)
            #print(ipage)
        else:
            return(notice_list)        
    else:
        return(notice_list)


In [36]:
# 목록의 내용만 크롤링 해옵니다.
# 목록의 링크 값을 가져와 suggestion_list 리스트에 담아줍니다.
notice_list = []
ipage = 1

get_notice_list(ipage)
notice_list


200
200


[['icon_ing.gif', '2020년 전자상거래 스타트업 스타일테크 지원사업 공고문', '2020-05-11', '34'],
 ['icon_ing.gif',
  '2020 Bu:Star 하이퍼 캐주얼 게임콘텐츠 제작지원 사업 공고',
  '2020-05-08',
  '248'],
 ['icon_ing.gif', '2020 Bu:Star 프로토타입 게임콘텐츠 제작지원 사업 공고', '2020-05-08', '239'],
 ['icon_ing.gif',
  '2020년 부산 VR/AR제작거점센터 VRAR융합콘텐츠 실증지원 사업 공고',
  '2020-05-06',
  '373'],
 ['icon_ing.gif', '게임콘텐츠 콘솔 컨버팅 지원사업 추가 모집 공고 ', '2020-05-04', '139'],
 ['icon_ing.gif',
  '[재공고] 2020년 일자리 르네상스 부산(제조혁신 엔지니어링 서비스 고용창출) ..',
  '2020-05-04',
  '330'],
 ['icon_ing.gif',
  '2020 게임콘텐츠 마케팅(글로벌_ 전략지원_ 런칭) 지원사업 통합공고',
  '2020-05-04',
  '328'],
 ['icon_ing.gif', '게임콘텐츠 글로벌 마케팅 지원사업 공고', '2020-04-28', '346'],
 ['icon_ing.gif', '「2020 유망기업 발굴 및 육성」용역 입찰 공고', '2020-04-28', '399'],
 ['icon_ing.gif',
  '2020 부산음악창작소 [컴필레이션] 음반제작 지원사업 뮤지션 모집 공고',
  '2020-04-24',
  '476'],
 ['icon_ing.gif',
  '2020 부산음악창작소 [정규/EP] 음반제작 지원사업 뮤지션 모집 공고',
  '2020-04-24',
  '819'],
 ['icon_end.gif',
  '2020 스마트 물류 및 ICT분야 상용화 지원사업 SW품질관리 협력업체 선정 결..',
  '2020-04-24',
  '343'],

In [38]:
with open('busanit_BusinessNotice1.csv', 'w') as file:    # busanit_BusinessNotice.csv 파일을 쓰기 모드로 열기
    file.write('status, title, reg_date, view_count\n')                       # 컬럼 이름 추가
    for i in notice_list:                                       # data를 반복하면서
        #print(i[1])
        file.write('{0},{1},{2},{3}\n'.format(i[0],i[1],i[2],i[3]))    # 등록일,사업공고제목을 줄 단위로 저장

In [39]:
# %matplotlib inline을 설정하면 matplotlib.pyplot의 show 함수를 호출하지 않아도
# 주피터 노트북 안에서 그래프가 표시됨
%matplotlib inline
import pandas as pd                # 데이터를 저장하고 처리하는 패키지
import matplotlib as mpl           # 그래프를 그리는 패키지
import matplotlib.pyplot as plt    # 그래프를 그리는 패키지

In [40]:
#print(len(suggestion_list))
#suggestion_list[:3]

In [59]:
# csv 파일을 읽어서 DataFrame 객체로 만듦. 인덱스 컬럼은 point로 설정 , encoding='CP949'
df = pd.read_csv('busanit_BusinessNotice1.csv', encoding='CP949')
df.head()    # df 표시
df.tail()
df.shape


(8, 4)

In [65]:
#c_df = df.copy()
#c_df
#df.dtypes
#df.describe()
#df.loc[df['status'] == 'icon_end.gif','status'] = 0
#df.loc[df['status'] == 'icon_ing.gif', 'status'] = 1
df

KeyError: 'title1'

In [56]:
# 불용어 - 자주 등장하지만 의미가 크지 않아 제외하고 싶은 텍스트
stopwords = ["지원사업", "2020년","공고","모집","지원사업","안내","제작지원","부산"]
#              "올해", "올해는", "열심히", "공부를", "합니다", "하고", "싶어요", 
#              "있는", "있습니다", "싶습니다", "2020년"]
# 불용어를 제거하지 않고 그리려면 아래 주석을 풀어서 stopword 변수에 덮어쓰기를 합니다.
# stopwords = []
# displayWordCloud 라는 함수를 만들어 재사용합니다.
from wordcloud import WordCloud


In [57]:
def displayWordCloud(data = None, backgroundcolor = 'black', width=1280, height=768):
    wordcloud = WordCloud(
                        font_path = 'C:/Windows/Fonts/malgun.ttf', 
                        stopwords = stopwords, 
                        background_color = backgroundcolor, 
                        width = width, height = height, 
                        random_state=42).generate(data)
    plt.figure(figsize = (15 , 10))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show() 

In [58]:
course_text = " ".join(df["title1"].fillna(''))
displayWordCloud(course_text)

KeyError: 'title1'